<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/Final_project/Larissa_Santesso/entrega_final/run02_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#!pip install import-ipynb
#import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#cd /content/gdrive/MyDrive/Colab\ Notebooks/Projeto/

/content/gdrive/.shortcut-targets-by-id/1OB8WtjF9IhjzAcbKVtlebYWYLazjwmVV/Projeto


In [ ]:
#!ls

'Apresentação de Projetos Finais - Larissa Santesso.gslides'
 arquiteturas_backup
 backup
 backup2
 checkpoints
 dataset
 modelos
 others
 run00_dataset.ipynb
 run01_metrics.ipynb
 run02_models.ipynb
 ViT_T5base_dsetComplete_exp001.ipynb


In [ ]:
from run00_dataset import *

# Model: Encoder (ViT) e Decoder (T5)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, pretrained_model):
        """
        Implements the Encoder."

        Args:
            pretrained_model (str): name of the pretrained model

        """

        super(Encoder, self).__init__()

        self.encoder = ViTModel.from_pretrained(pretrained_model)

    def forward(self, input):
        out  = self.encoder(pixel_values = input)

        return out

In [ ]:
class Decoder(nn.Module):
    def __init__(self, pretrained_model, encoder_modeldim):
        """
        Implements the Encoder."

        Args:
            pretrained_model (str): name of the pretrained model

        """
        
        super(Decoder, self).__init__()
        
        self.decoder = T5ForConditionalGeneration.from_pretrained(pretrained_model)
        self.linear = nn.Linear(self.decoder.model_dim, encoder_modeldim, bias = False)
        self.encoder_modeldim = encoder_modeldim

    def forward(self, output_encoder, targets, decoder_ids=None):

        if self.decoder.model_dim!=self.encoder_modeldim:
            print(f"Changed model hidden dimension from {self.encoder_modeldim} to {self.decoder.model_dim}")
            output_encoder = self.linear(output_encoder)
            print(output_encoder.shape)

        # Validation/Testing
        if decoder_ids is not None:
            out = self.decoder(encoder_outputs=output_encoder, decoder_input_ids=decoder_ids)

        # Training
        else:
            out = self.decoder(encoder_outputs=output_encoder, labels=targets)

        return out

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, pretrained_model: Tuple[str], encoder_dmodel=768, eos_token_id=None, pad_token_id=None):
        """
        Implements a model that combines MyEncoder and MyDecoder."

        Args:
            pretrained_model (tuple): name of the pretrained model
            encoder_dmodel (int):  hidden dimension of the encoder output
            eos_token_id (torch.long): token used for end of sentence  
            pad_token_id (torch.long): token used for padding

        """
        
        super(EncoderDecoder, self).__init__()
        self.eos_token_id = eos_token_id
        self.pad_token_id = pad_token_id
        self.encoder = Encoder(pretrained_model[0])
        self.encoder_dmodel = encoder_dmodel

        # Freeze parameters from encoder
        #for p in self.encoder.parameters():
        #    p.requires_grad=False

        self.decoder = Decoder(pretrained_model[1], self.encoder_dmodel)
        self.decoder_start_token_id = self.decoder.decoder.config.decoder_start_token_id

    def forward(self, images = None, targets = None, decoder_ids = None):
        output_encoder  = self.encoder(images)
        out = self.decoder(output_encoder, targets, decoder_ids)

        return out

In [ ]:
# Uncomment if you want to check model parameters
# These lines were comemmented to avoid waste of memory

#model = EncoderDecoder(pretrained_model = (config_exp["encoder_pretrained_model"], config_exp["decoder_pretrained_model"]), 
#                       eos_token_id=tokenizer.eos_token_id, 
#                       pad_token_id=tokenizer.pad_token_id)

In [ ]:
#num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
#print(f'Número de parâmetros do modelo: {num_params}')

Número de parâmetros do modelo: 309882624
